# 預測資料處理

>載入讀取資料的套件

In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

>讀取預測資料

In [2]:
busfactor = pd.read_csv('testdata.csv',encoding='BIG5')
busfactor.drop('Unnamed: 0', axis = 1, inplace = True)

In [3]:
busfactor.head()

,日期,班次,StopName_Zh_tw,StopSequence,UP,DOWN,PASSNUM,pedict_up,pedict_down,pedict_passnum,drive_passnum
0,2019/7/1,900,前鎮站,1,1,0,1,3,0,3,3
1,2019/7/1,900,輕軌夢時代站(統一時代百貨),2,0,0,1,0,0,2,3
2,2019/7/1,900,台糖物流,3,0,0,1,0,0,2,3
3,2019/7/1,900,輕軌經貿園區站,4,0,0,1,0,0,2,3
4,2019/7/1,900,正勤路,5,1,0,2,2,0,4,5


>將預測資料轉成模式參數

In [4]:
#旅行時間
timeable={'前鎮站':'800','輕軌夢時代站(統一時代百貨)':'801','台糖物流':'802','輕軌經貿園區站':'803','正勤路':'803'
          ,'正勤社區':'804','中華復興路口':'805','中華五路':'806','新光路口(圖書總館)':'807','三多四路口':'808'
         ,'大遠百百貨(捷運三多商圈站)':'810','新光三越(捷運三多商圈站)':'811','三多三路':'812','仁愛三街口':'813'
         ,'三多復興路口':'814','三多市場(三多三路)':'815','光華路口(光華夜市)':'816','林德官(三和市場)':'816'
         ,'和平路口':'817','五權國小(三多二路)':'818','凱旋二路口':'818','國際商工':'819','三信家商':'820'
         ,'五塊厝(三多一路)':'821','輔仁路口(三多一路)':'822','中正體育場':'823','衛武營':'824','衛武營國家藝術文化中心':'825'
         ,'衛武社區':'828','鳳山行政中心(澄清路)':'830','覺民路口(澄清路)':'832','褒揚街口':'833','澄清路':'835'
         ,'大華村':'836','正修科大(澄清湖)':'838','長庚復健大樓':'844','長庚醫院':'846','澄清湖棒球場':'848'}
time=timeable.values()

###### 整合投入資料

>1. 投入：預測上車人數、下車人數、站間量

In [5]:
#7/1資料
m1=dict()
for i,j in enumerate(busfactor['日期']):
    if j == '2019/7/1':
        stop = busfactor['StopName_Zh_tw'][i]
        m1[stop]=dict()
        m1[stop]['pedict_up'] =busfactor['pedict_up'][i]
        m1[stop]['pedict_down'] =busfactor['pedict_down'][i]
        if busfactor['drive_passnum'][i] < 0:
            m1[stop]['drive_passnum'] =busfactor['pedict_passnum'][i]
        else:
            m1[stop]['drive_passnum'] =busfactor['drive_passnum'][i]

In [6]:
stop=[]
for i in m1:
    stop.append(i)

>2.投入:旅行時間

In [7]:
for i,j in enumerate(stop):
    if i == 0:
        m1[j]['travel_time']= 0
    else:
        m1[j]['travel_time']= int(timeable[j])-int(timeable[stop[i-1]])

>3.投入：抵達率

In [8]:
#班距
ex=20
#抵達率(人/分鐘)
for i in m1:
    m1[i]['arrival_rate']=m1[i]['pedict_up']/ex
#四捨五入round()

In [9]:
m1

{'前鎮站': {'pedict_up': 3,
  'pedict_down': 0,
  'drive_passnum': 3,
  'travel_time': 0,
  'arrival_rate': 0.15},
 '輕軌夢時代站(統一時代百貨)': {'pedict_up': 0,
  'pedict_down': 1,
  'drive_passnum': 2,
  'travel_time': 1,
  'arrival_rate': 0.0},
 '台糖物流': {'pedict_up': 0,
  'pedict_down': 0,
  'drive_passnum': 2,
  'travel_time': 1,
  'arrival_rate': 0.0},
 '輕軌經貿園區站': {'pedict_up': 1,
  'pedict_down': 0,
  'drive_passnum': 3,
  'travel_time': 1,
  'arrival_rate': 0.05},
 '正勤路': {'pedict_up': 1,
  'pedict_down': 0,
  'drive_passnum': 4,
  'travel_time': 0,
  'arrival_rate': 0.05},
 '正勤社區': {'pedict_up': 1,
  'pedict_down': 1,
  'drive_passnum': 4,
  'travel_time': 1,
  'arrival_rate': 0.05},
 '中華復興路口': {'pedict_up': 0,
  'pedict_down': 1,
  'drive_passnum': 3,
  'travel_time': 1,
  'arrival_rate': 0.0},
 '中華五路': {'pedict_up': 0,
  'pedict_down': 0,
  'drive_passnum': 3,
  'travel_time': 1,
  'arrival_rate': 0.0},
 '新光路口(圖書總館)': {'pedict_up': 0,
  'pedict_down': 0,
  'drive_passnum': 3,
  'travel_tim

## 模型

>載入加班車模式套件

In [10]:
from gurobipy import *
import math

In [11]:
#預測45分鐘，因為許多站間的旅行時間為一分鐘，甚至不到，因此在路往時間軸以1分鐘為一單位。
T=45
delta=1
#set K M B VA VN MTA
#觀測站點(K)
K=stop
#建立兩大路網中的node
demand=[]
for m in m1:
    for l in range(1,T+1):
        demand.append((m,l))
#原班次的班距(分鐘)
schedule=15
#原班次的起始時間
st=[]
for i in range(T):
    if i%schedule == 0:
        st.append(i)

In [12]:
#建立乘客路網的arc(B)
B=[]
for i in demand:
    tt = i[1]
    a=K.index(i[0])
    b=K.index(K[-1])
    t=i[1]
    for l in range(a,b):
        t+= m1[K[l+1]]['travel_time']
        if t <= T:
            B.append(((K[l],tt),(K[l+1],t)))
        elif tt <= T:
            B.append(((K[l],tt),'dummy'))
        tt = t
for m in demand:
    B.append(('source',m))
    B.append((m,'sink'))
for t in range(1,T+1):
        B.append((('%s'%(stop[-1]),t),'dummy'))

In [13]:
#建立公車路網的所有arc(VV)&公車屬於加班車的載客arc(VAB)&固定班次的載客arc(SVS)
VV=tuplelist()
VAB=tuplelist()
SVS=tuplelist()
k=stop[0]
for i in range(1,T+1):
    VV.append(('source',(k,i)))     
VV.append(('source','sink'))
for i in range(1,T+1):
    for m,l in enumerate(K):
        if i != T and K[m] != K[-1]:
            if i+m1[K[m+1]]['travel_time'] <= T:
                VV.append(((l,i),(K[m+1],i+m1[K[m+1]]['travel_time'])))
                #檢查是否為原班次
                for check in st:
                    for cc in range(K.index(l)+1):
                        check+= m1[K[cc]]['travel_time']
                    if i != check:
                        VAB.append(((l,i),(K[m+1],i+m1[K[m+1]]['travel_time'])))
                    else :
                        SVS.append(((l,i),(K[m+1],i+m1[K[m+1]]['travel_time'])))
            else:
                VV.append(((l,i),'sink'))
for k in K:
    VV.append(((k,T),'sink')) 
#VS & VD
VS=['source']
VD=['sink']            
#VN       
VN=[]
for k in K:
    for t in range(1,T+1):
        VN.append((k,t))
#MN
MN=[]
for i in VAB:
    MN.append(i[0])
    MN.append(i[1])
MN=list(set(MN))

In [14]:
#param
rev=-16
km= 5
cc=38*km        
e=dict()
e=dict()
for k,j in enumerate(demand):
    e[j]=m1[j[0]]['arrival_rate']
        
q=10
veh=3

In [15]:
#variable
m = Model("realcontrol_6.0")
#m.setParam('MIPGAP',0.5)
x = {}
for i in VV:
    x[i[0],i[1]] = m.addVar(vtype = GRB.INTEGER,lb=0, name = "x[%s,%s]" %(i[0],i[1]))
y = {}
for i in B:
    y[i[0],i[1]] = m.addVar(vtype = GRB.CONTINUOUS,lb=0, name = "y[%s,%s]" %(i[0],i[1]))

Using license file C:\Users\James\gurobi.lic
Academic license - for non-commercial use only


In [19]:
#obj
m.setObjective(sum((sum(y[a,h] for a,h in B if a ==j  and h != 'sink' and h != 'dummy' and a[0] != h[0] )\
            -sum(y[i,b] for i,b in B if b == j and i[0] != b[0]\
            and i != 'source'  and b != 'sink'and b[1] != T)) \
            for j in MN )*rev +\
            sum(cc*x[i[0],i[1]] for i in VV if i[0] == 'source' and i[1] != 'sink')  , GRB.MINIMIZE)

#subjectto
for i in VS:
    m.addConstr(quicksum(x[i,j] for i,j in VV.select(i,'*')) == veh)
    
for j in VD:
    m.addConstr(quicksum(x[i,j] for i,j in VV.select('*',j)) == veh)

for i in VN:
    m.addConstr(quicksum(x[h,j] for h,j in VV if h == i) - quicksum(x[j,l] for j,l in VV if l == i) == 0)
    

for i in MN:
    if i[1] <= T:
        m.addConstr(quicksum(y[h,j] for h,j in B if h==i  ) - quicksum(y[j,l] for j,l in B if l == i) == 0)

for a,k in enumerate(demand):
    m.addConstr(y['source',k] == e[k]*(k[1]%15))

for i in SVS:
    if i[0][1] <= T:
        if i[0][0] != i[1][0]:
            m.addConstr(y[i[0],i[1]]  <= q*x[i[0],i[1]])

for i in B:
    if i[0][0] == i[1][0]:
        m.addConstr(9.0*y[i[0],'sink'] == y[i[0],i[1]])

In [20]:
m.optimize()
for v in m.getVars():
    if v.x > 0.001:
        print ('%s:%f' %(v.varName,v.x))
print ('objective value:%f'  %(m.objVal))

Gurobi Optimizer version 9.0.0 build v9.0.0rc2 (win64)
Optimize a model with 10408 rows, 27325 columns and 24032 nonzeros
Model fingerprint: 0x7c11199c
Variable types: 25613 continuous, 1712 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+01]
  Objective range  [2e+01, 6e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e-02, 3e+00]

MIP start from previous solve produced solution with objective -4657.6 (0.18s)
Loaded MIP start from previous solve with objective -4657.6

Presolve removed 10408 rows and 27325 columns
Presolve time: 0.04s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.23 seconds
Thread count was 1 (of 12 available processors)

Solution count 2: -4657.6 
No other solutions better than -4657.6

Optimal solution found (tolerance 1.00e-04)
Best objective -4.657600000000e+03, best bound -4.657600000000e+03, gap 0.0000%
x[source,sink]:3.000000
y[('前鎮站', 1),('輕軌夢時代站(統一時代百貨)', 2)]:0.150000
y[('前鎮站', 2),('輕軌

y[source,('仁愛三街口', 7)]:0.700000
y[source,('仁愛三街口', 8)]:0.800000
y[source,('仁愛三街口', 9)]:0.900000
y[source,('仁愛三街口', 10)]:1.000000
y[source,('仁愛三街口', 11)]:1.100000
y[source,('仁愛三街口', 12)]:1.200000
y[source,('仁愛三街口', 13)]:1.300000
y[('仁愛三街口', 13),sink]:1.300000
y[source,('仁愛三街口', 14)]:1.400000
y[source,('仁愛三街口', 16)]:0.100000
y[source,('仁愛三街口', 17)]:0.200000
y[source,('仁愛三街口', 18)]:0.300000
y[source,('仁愛三街口', 19)]:0.400000
y[source,('仁愛三街口', 20)]:0.500000
y[source,('仁愛三街口', 21)]:0.600000
y[source,('仁愛三街口', 22)]:0.700000
y[source,('仁愛三街口', 23)]:0.800000
y[source,('仁愛三街口', 24)]:0.900000
y[source,('仁愛三街口', 25)]:1.000000
y[source,('仁愛三街口', 26)]:1.100000
y[source,('仁愛三街口', 27)]:1.200000
y[('仁愛三街口', 27),sink]:7.600000
y[source,('仁愛三街口', 28)]:1.300000
y[('仁愛三街口', 28),sink]:1.300000
y[source,('仁愛三街口', 29)]:1.400000
y[source,('仁愛三街口', 31)]:0.100000
y[('仁愛三街口', 31),sink]:5.550000
y[source,('仁愛三街口', 32)]:0.200000
y[('仁愛三街口', 32),sink]:4.100000
y[source,('仁愛三街口', 33)]:0.300000
y[source,('仁愛三街口', 34)]

y[('輔仁路口(三多一路)', 12),sink]:0.550000
y[('輔仁路口(三多一路)', 13),sink]:0.600000
y[('輔仁路口(三多一路)', 15),sink]:0.700000
y[('輔仁路口(三多一路)', 16),sink]:0.700000
y[('輔仁路口(三多一路)', 17),sink]:0.050000
y[('輔仁路口(三多一路)', 18),sink]:0.100000
y[('輔仁路口(三多一路)', 20),sink]:0.200000
y[('輔仁路口(三多一路)', 24),sink]:0.400000
y[('輔仁路口(三多一路)', 25),sink]:0.450000
y[('輔仁路口(三多一路)', 26),sink]:0.500000
y[('輔仁路口(三多一路)', 27),sink]:0.550000
y[('輔仁路口(三多一路)', 28),sink]:0.600000
y[('輔仁路口(三多一路)', 30),sink]:0.700000
y[('輔仁路口(三多一路)', 33),sink]:0.100000
y[('輔仁路口(三多一路)', 36),sink]:0.250000
y[('輔仁路口(三多一路)', 38),sink]:0.350000
y[('輔仁路口(三多一路)', 40),sink]:0.450000
y[('輔仁路口(三多一路)', 41),sink]:0.500000
y[('輔仁路口(三多一路)', 45),sink]:11.400000
y[source,('中正體育場', 1)]:0.050000
y[source,('中正體育場', 2)]:0.100000
y[source,('中正體育場', 3)]:0.150000
y[source,('中正體育場', 4)]:0.200000
y[source,('中正體育場', 5)]:0.250000
y[source,('中正體育場', 6)]:0.300000
y[source,('中正體育場', 7)]:0.350000
y[source,('中正體育場', 8)]:0.400000
y[source,('中正體育場', 9)]:0.450000
y[('中正體育場', 9),sink]:0.4500